In [19]:
import requests
from bs4 import BeautifulSoup
from astropy.io import ascii
import pandas as pd
import concurrent.futures
import glob
from time import sleep
import os
import numpy as np
from matplotlib import pyplot as plt

In [20]:
header = {
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
'Referer': 'http://irsa.ipac.caltech.edu/ibe/docs/ptf/images/level1/',
'Cookie': '__utma=253438417.1413587276.1479455055.1486991316.1487038069.11; __utmz=253438417.1479459352.2.2.utmcsr=yahoo|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); usrkey="276c187d-93a1-4d19-936e-edc27049cc5b/ncuiarex@astro.ncu.edu.tw"; __utmc=253438417; JOSSO_REMEMBERME_josso=mUXCaMhYWY4Hn9io8eFhGwlHfCVop7quyI8e%2FsSJJp8%3D; JOSSO_SESSIONID=B5A244740F815B1D5A9C1C3AB431DCD0'
}    

In [21]:
#os.chdir('D:\kepler_ptf_research\k2_c4')
os.chdir('/media/careyjohn/DATA/kepler_ptf_research/k2_c4')

In [22]:
df = pd.read_csv('k2_c4_m-dwarfs.csv', skiprows=1)
# df.head()
RA = df['ra']
DEC = df['dec']

In [7]:
df['check_band'] = -99

In [8]:
# c = progressbar.ProgressBar(maxval=len(df), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

def check_band(ra, dec):
    ind = get_index(ra)
    if len(glob.glob('index/{}*.txt'.format(ind))) == 0:
        res = requests.get("http://irsa.ipac.caltech.edu/ibe/search/ptf/images/level1?POS={0},{1}".format(ra, dec), headers=header)
        soup = BeautifulSoup(res.text, 'lxml')
        data = ascii.read(soup.text)
        a = data['filter']
        ha656 = a[a == "HA656"]
        ha663 = a[a == "HA663"]
        if len(ha656) >= 1 and len(ha663) >= 1:
            f = open('index/{}_1.txt'.format(ind), 'w')
            f.close()
        else:
            f = open('index/{}_0.txt'.format(ind), 'w')
            f.close()
        return None
    else:
        return None

In [12]:
def get_index(ra):
    return RA[RA == ra].index[0]

In [22]:
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(check_band, RA, DEC)

In [23]:
os.getcwd()

'D:\\kepler_ptf_research\\k2_c4'

In [20]:
len(RA)

2783

In [24]:
len(DEC)

2783

In [23]:
cd index

/media/careyjohn/DATA/kepler_ptf_research/k2_c4/index


In [24]:
name = 'both_ha_have.txt'

filej = open(name)
data = np.loadtxt( filej )
index = list(map(int, data))

In [25]:
RA_H = list(RA[index[0:len(index)-1]])
DEC_H = list(DEC[index[0:len(index)-1]])

len(RA_H)

2718

In [26]:
plt.figure(figsize=(10,8))
plt.plot(RA_H, DEC_H, 'bo', markersize=1.)
plt.title('2718 M dwarfs have ptf H$a$656&663 on Kpler K2 Campaign 4')
#plt.plot(ra, dec, 'ro', markersize=1.5)
plt.grid(True)
plt.xlim(49, 68)
plt.ylim(min(DEC_H)-1, max(DEC_H)+1)
plt.xlabel('$RA  (deg)$', size = 15)
plt.ylabel('$Dec  (deg)$', size = 15)
plt.show()

In [36]:
df2 = pd.read_csv('k2_c4_m-dwarfs.csv')

In [37]:
m_ty_epic = df2['EPIC']

In [38]:
len(m_ty_epic)

2783

In [40]:
m_ty_epic_H = list(m_ty_epic[index[0:len(index)-1]])

In [42]:
len(m_ty_epic_H)

2718

In [43]:
np.savetxt("m-dwarfs_ptf-in-k2c4.txt", m_ty_epic_H, fmt= '%i')

In [44]:
pwd

u'/media/careyjohn/DATA/kepler_ptf_research/k2_c4'

In [48]:
df3 = pd.read_csv('ptf_ha_substraction_k2c4.csv')

In [49]:
len(df3['EPIC'])

2718